In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

toxicmender_20bn_jester_path = kagglehub.dataset_download('toxicmender/20bn-jester')
isharau_trimmed_path = kagglehub.dataset_download('isharau/trimmed')

print('Data source import complete.')

In [ ]:
!find / -name "20bn-jester" 2>/dev/null

In [ ]:
!ls /root/.cache/kagglehub/datasets/toxicmender/20bn-jester/versions/3

In [ ]:
!cp -r /root/.cache/kagglehub/datasets/toxicmender/20bn-jester /content/  # run this if you want to copy jester to here

## **CNN-LSTM**

In [5]:
import os
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tqdm import tqdm
import matplotlib.pyplot as plt

In [7]:
def load_data(folder_path, csv_path, is_train=True, max_frames=30, img_size=(64, 64)):
    df = pd.read_csv(csv_path)
    video_ids = df['video_id'].values
    labels = df['label'].values if is_train else None

    num_samples = len(video_ids)
    data = np.zeros((num_samples, max_frames, img_size[0], img_size[1], 3), dtype=np.uint8)
    label_list = []

    for i, video_id in tqdm(enumerate(video_ids), total=num_samples, desc="Loading images"):
        video_folder = os.path.join(folder_path, str(video_id))
        if not os.path.exists(video_folder):
            print(f"Warning: Folder {video_folder} not found.")
            continue

        frames = []
        for img_name in sorted(os.listdir(video_folder))[:max_frames]:
            img_path = os.path.join(video_folder, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, img_size)
                frames.append(img)

        frames = np.array(frames, dtype=np.uint8)
        data[i, :len(frames)] = frames

        if is_train:
            label_list.append(labels[i])

    if is_train:
        return data, np.array(label_list)
    else:
        return data, None

In [ ]:
train_folder = "/root/.cache/kagglehub/datasets/toxicmender/20bn-jester/versions/3/Train"
test_folder = "/root/.cache/kagglehub/datasets/toxicmender/20bn-jester/versions/3/Test"
train_csv = "/root/.cache/kagglehub/datasets/toxicmender/20bn-jester/versions/3/Train.csv"
test_csv = "/content/Test.csv"

X_train, y_train = load_data(train_folder, train_csv, is_train=True)
X_test, _ = load_data(test_folder, test_csv, is_train=False)

# Encode labels
unique_labels = np.unique(y_train)
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
y_train_encoded = np.array([label_mapping[label] for label in y_train])
y_train_onehot = to_categorical(y_train_encoded, num_classes=len(unique_labels))


In [ ]:
# Normalize pixel values
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Ensure the shape is consistent
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 64, 64, 3))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 64, 64, 3))


### Model

In [ ]:
model = models.Sequential([
    # CNN Layers
    layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu'), input_shape=(None, 64, 64, 3)),
    layers.TimeDistributed(layers.MaxPooling2D((2, 2))),
    layers.TimeDistributed(layers.Conv2D(128, (3, 3), activation='relu')),
    layers.TimeDistributed(layers.MaxPooling2D((2, 2))),
    layers.TimeDistributed(layers.GlobalAveragePooling2D()),  # Reduces parameters compared to Flatten

    # LSTM Layers
    layers.LSTM(64, return_sequences=True, activation='tanh'),
    layers.LSTM(32, activation='tanh'),

    # Dense Layers
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(unique_labels), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

In [ ]:
# Training
history = model.fit(X_train, y_train_onehot, epochs=50, batch_size=8, validation_split=0.2)

In [ ]:
predictions = model.predict(X_test)
predicted_labels = [unique_labels[np.argmax(pred)] for pred in predictions]

test_df = pd.read_csv(test_csv)
test_df['label'] = predicted_labels
test_df.head()

In [ ]:
import matplotlib.pyplot as plt

# Plot akurasi
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training Accuracy')
plt.show()

# Plot loss
plt.plot(history.history['loss'], label='Train Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training Loss')
plt.show()

In [ ]:
final_accuracy = history.history['accuracy'][-1]
print(f"Final Training Accuracy: {final_accuracy * 100:.2f}%")

for epoch, acc in enumerate(history.history['accuracy'], start=1):
    print(f"Epoch {epoch}: Training Accuracy = {acc * 100:.2f}%")

final_loss = history.history['loss'][-1]
print(f"Final Training Loss: {final_loss:.4f}")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

# Plot loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

## **CNN-GRU**

In [ ]:
model = models.Sequential([
    # CNN Layers
    layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu'), input_shape=(None, 64, 64, 3)),
    layers.TimeDistributed(layers.MaxPooling2D((2, 2))),
    layers.TimeDistributed(layers.Conv2D(128, (3, 3), activation='relu')),
    layers.TimeDistributed(layers.MaxPooling2D((2, 2))),
    layers.TimeDistributed(layers.GlobalAveragePooling2D()),  # Reduces parameters compared to Flatten

    # GRU Layers
    layers.GRU(64, return_sequences=True, activation='tanh'),
    layers.GRU(32, activation='tanh'),

    # Dense Layers
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(unique_labels), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

In [ ]:
# Training the model
history = model.fit(X_train, y_train_onehot, epochs=50, batch_size=4, validation_split=0.2)

In [ ]:
# Predictions on test data
predictions = model.predict(X_test)
predicted_labels = [unique_labels[np.argmax(pred)] for pred in predictions]

test_df = pd.read_csv(test_csv)
test_df['label'] = predicted_labels
test_df.head()

In [ ]:
# Plot Training and Validation Accuracy
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

In [ ]:
# Plot Training and Validation Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Print final metrics
final_accuracy = history.history['accuracy'][-1]
final_loss = history.history['loss'][-1]
print(f"Final Training Accuracy: {final_accuracy * 100:.2f}%")
print(f"Final Training Loss: {final_loss:.4f}")